# Prompt Engineering and Prompting Techniques

Prompt engineering involves designing and refining input prompts to effectively communicate with language models, optimizing their performance on specific tasks. By carefully crafting these prompts, users can guide models to produce more accurate and relevant outputs, enhancing their utility in applications like chatbots and content generation.

## Lab Description:

This lab explores effective prompting techniques for Large Language Models. It starts with basic text completion, text summarization and sentiment analysis. Participants will then experiment with code genera|tion by prompting the LLM to generate neat and correct code both in Python and SQL. Finally, the lab covers advanced prompting techniques, including few-shot and chain-of-thought prompting.

## Lab Objectives:

After completion of the lab, the participants will be able to:

- Effectively construct prompts for basic text completion, sentiment analysis, code generation, and text summarization.
  
- Prompt the LLM to write neat and correct Code.
  
- Utilize advanced prompting techniques, including few-shot and chain-of-thought prompting, to improve model accuracy and reasoning.
  

## Importing the Libraries

This lab requires the following libraries:

- `langchain_community` : This library contains the `Ollama` method which we leverage to connect to the LLM hosted on Ollama.
- `langchain_core` : This library contains the methods like `StrOutputParser` , `HumanMessage` and `SystemMessage`.

In [1]:
from langchain_community.llms import Ollama                     #Importing the Ollama library for langchain
from langchain_core.output_parsers import StrOutputParser       #Importing the stringoutput parser
from langchain_core.messages import HumanMessage, SystemMessage #Langchain messages class provides a way to communicate with the LLM, The HumanMessage corresponds to the human message string (like the questions to the llm) and the SystemMessages corresponds to the instructions to the LLM

In [29]:
model = Ollama(model = "llama3.1:8b", base_url="http://10.79.253.112:11434", temperature=0.0)   # Loading the ollama model

## Basic Chat Completion

Basic chat completion involves providing a prompt to a Large Language Model (LLM) for making it predict the next word in a sentence. The model completes the given sentence in a conversational manner by analyzing context and language patterns. This forms the foundation for more advanced prompting techniques.

In [3]:
messages = [
    SystemMessage(
        content="Complete the given sentence in one word"   #Instruction to the LLM
    ),
    HumanMessage(
        content="The sky is"                                #The human Question 
    )
]

response = model.invoke(messages)                           #Invokes the chain with the message we designed
response

'Blue.'

This can also be done using prompt templates. Langchain provides a prompttemplate class

In [20]:
from langchain.prompts import PromptTemplate
from IPython.display import Markdown, display

## Components of a Basic Prompt Template

A basic prompt template usually consists of and instruction message followed by a Placeholder. The Placeholder can be anything varying from text that needs to be summarized, retrieved documents and so on.

<div style="text-align: center;">
    <img src="prompt_template.png" alt="flow" width="700" height="450">
</div>

In [8]:
prompt = PromptTemplate.from_template(               #The .from_template() method loads a prompt template from a template(in this case, the string we provided)
    """Complete the following sentence in one word      

       sentence: {sentence}"""
)

sentence = "The sky is"                              #The sentence to format the prompt

chain = prompt | model | StrOutputParser()           #Building the chain

response = chain.invoke({"sentence":sentence})       #Invoking the chain
response

'Blue.'

## Text Summarization

LLMs are widely used for text summarization tasks. This section introduces prompts that could efficiently guide the LLMs to summarize a given chunk of text.

In [10]:
messages = [
    SystemMessage(
        content="Summarize the given context in 2 sentences at most"   #Instruction to the LLM
    ),
    HumanMessage(
        content= "Generative AI refers to a class of artificial intelligence techniques that can create new content, such as text, images, music, and more, by learning patterns from existing data. These models, like GPT and DALL-E, utilize deep learning algorithms to generate outputs that can mimic human creativity and style. The applications of generative AI are vast, ranging from content creation and virtual art to drug discovery and game design. As this technology continues to evolve, it raises important questions about originality, ethics, and the potential for misuse. Overall, generative AI is transforming various industries by enabling new forms of creativity and automation."
                    
    )
]

response = model.invoke(messages)
response

'Here is a 2-sentence summary:\n\nGenerative AI refers to artificial intelligence techniques that can create new content such as text, images, music, and more by learning patterns from existing data. These models have vast applications in various industries, but also raise important questions about originality, ethics, and potential misuse.'

## Question Answering

LLMs are quite good at Question Answering. This section demonstrates prompts that could efficiently make LLMs generate meaningful, accurate and concise answers to user questions.

In [11]:
# Create a prompt template to guide the model's responses based on a provided context
prompt = PromptTemplate.from_template(
    """Answer the question based on the context below. Be short and concise. Reply "Unsure" if you are not sure about the answer. Donot make up answers.

       Context: Prompt engineering is the process of designing input prompts to effectively communicate with language models, optimizing their outputs for specific tasks. By carefully crafting the wording, structure, and context of prompts, users can enhance the accuracy and relevance of the model's responses. This technique is crucial in applications like chatbots and content generation, where the quality of the input directly influences the quality of the generated content.

       Question: {question}"""
)

# Define the question to be asked based on the context
question = "How can prompt engineers enhance the quality of the model's response?"

# Create a chain that combines the prompt with the model and parser for the response
chain = prompt | model | StrOutputParser()

# Invoke the chain with the question and store the response
response = chain.invoke({"question": question})

# Output the response
print(response)


'By carefully crafting the wording, structure, and context of prompts to effectively communicate with language models.'

## Sentiment Analysis

This section demonstrates prompting to make the model classify a sentence as positive, neutral or negative.

In [13]:
# Create a prompt template to classify a given sentence as positive, neutral, or negative
prompt = PromptTemplate.from_template(
    """Classify the following sentence into positive, neutral or negative. Do not explain how you arrived at the answer.

       Sentence: {sentence}"""
)

# Define the sentence to be classified
sentence = "The burgers were really tasty."

# Create a chain that combines the prompt with the model and parser for the response
chain = prompt | model | StrOutputParser()

# Invoke the chain with the sentence and store the response
response = chain.invoke({"sentence": sentence})

# Output the response
print(response)


'Positive'

#### Trying with a negative sentence. 

In [14]:
# Create a prompt template to classify a given sentence as positive, neutral, or negative
prompt = PromptTemplate.from_template(
    """Classify the following sentence into positive, neutral or negative. Do not explain how you arrived at the answer.

       Sentence: {sentence}"""
)

# Define the sentence to be classified
sentence = "The service was not upto the mark"

# Create a chain that combines the prompt with the model and parser for the response
chain = prompt | model | StrOutputParser()

# Invoke the chain with the sentence and store the response
response = chain.invoke({"sentence": sentence})

# Output the response
print(response)


'Negative.'

## CODE GENERATION

### SQL QUERY

Here, we provide the LLM with a table and an instruction to generate a SQL Query based on that table.

In [22]:
# Create a list of messages for the model to process
messages = [
    # System message providing instructions to the LLM
    SystemMessage(
        content="Create a SQL query to find the names and salaries of all employees who work in the 'Sales' department, with respect to the table details given. Generate only the query, no explanation is needed."
    ),
    # Human message providing the context and table details
    HumanMessage(
        content= """
                 Table Employees, columns = [EmployeeId, FirstName, LastName, DepartmentId, Salary]
                 Table Departments, columns = [DepartmentId, DepartmentName]
                 """                
    )
]

# Invoke the model with the messages and store the response
response = model.invoke(messages)

# Display the generated SQL query as formatted Markdown
display(Markdown(response))


```sql
SELECT E.FirstName, E.LastName, D.DepartmentName, E.Salary
FROM Employees E
JOIN Departments D ON E.DepartmentId = D.DepartmentId
WHERE D.DepartmentName = 'Sales';
```



### PYTHON

Here we provide a simple code generation prompt to make the LLM generate a python function.

In [21]:
# Create a prompt template to generate a Python method based on the provided question
prompt = PromptTemplate.from_template(
    """Generate python method for the asked question. Provide the code with comments. Do not explain the code
    
       Question: {question}"""
)

# Define the question to be asked
question = "Reverse a string"

# Create a chain that combines the prompt with the model and parser for the response
chain = prompt | model | StrOutputParser()

# Invoke the chain with the question and store the response
response = chain.invoke({"question": question})

# Display the generated Python code as formatted Markdown
display(Markdown(response))



```python
def reverse_string(input_str):
    """
    This function takes an input string and returns its reversed version.
    
    Parameters:
    input_str (str): The string to be reversed.
    
    Returns:
    str: The reversed string.
    """
    
    # Initialize an empty string to store the reversed string
    reversed_str = ""
    
    # Iterate over each character in the input string
    for char in input_str:
        # Add each character at the beginning of the reversed string (equivalent to reversing)
        reversed_str = char + reversed_str
    
    # Return the reversed string
    return reversed_str

# Test the function with a sample string
input_string = "Hello, World!"
print("Original String:", input_string)
print("Reversed String:", reverse_string(input_string))
```

## ADVANCED PROMPTING TECHNIQUES

### Few-Shot Prompts

Few-shot prompts are a technique in natural language processing that provides a model with a small number of examples (or "shots") to guide its responses for a specific task. This approach allows the model to learn from the provided examples and generate outputs that align with the desired format or content, improving its performance on tasks with limited training data.

In [31]:
# Define a few-shot prompt containing multiple questions and answers about sums of multiples of 3
prompt = """Q: The sum of the multiples of 3 in this group is greater than 20: 3, 7, 9, 12, 5, 14.
            Answer:True

            Q: The sum of the multiples of 3 in this group is greater than 20: 6, 4, 9, 15, 2, 18.
            Answer:True

            Q: The sum of the multiples of 3 in this group is greater than 20: 1, 2, 4, 6, 3, 10.
            Answer:False

            Q: The sum of the multiples of 3 in this group is greater than 20: 9, 18, 5, 7, 21, 12.
            Answer:True

            Q: The sum of the multiples of 3 in this group is greater than 20: 8, 3, 14, 27, 5, 6.
            Answer: """  # Few-shot prompt providing context for the model

# Prepare the message for the model
messages = [
    {
        "role": "user",
        "content": prompt
    }
]

# Invoke the model with the messages and store the response
response = model.invoke(messages)

# Display the response as formatted Markdown
display(Markdown(response))


It looks like you have a series of questions about whether the sum of the multiples of 3 in each group is greater than 20. I'll answer them for you.

Q: The sum of the multiples of 3 in this group is greater than 20: 3, 7, 9, 12, 5, 14.
Answer: True

The multiples of 3 in this group are 3, 9, and 12. Their sum is 24, which is indeed greater than 20.

Q: The sum of the multiples of 3 in this group is greater than 20: 6, 4, 9, 15, 2, 18.
Answer: True

The multiples of 3 in this group are 6, 9, and 18. Their sum is 33, which is indeed greater than 20.

Q: The sum of the multiples of 3 in this group is greater than 20: 1, 2, 4, 6, 3, 10.
Answer: False

The multiples of 3 in this group are 3 and 6. Their sum is 9, which is not greater than 20.

Q: The sum of the multiples of 3 in this group is greater than 20: 9, 18, 5, 7, 21, 12.
Answer: True

The multiples of 3 in this group are 9, 18, and 21. Their sum is 48, which is indeed greater than 20.

Q: The sum of the multiples of 3 in this group is greater than 20: 8, 3, 14, 27, 5, 6.
Answer: True

The multiples of 3 in this group are 3, 6, and 27. Their sum is 36, which is indeed greater than 20.

### Chain-Of-Thought Prompting

Chain of thought prompting is a technique in natural language processing that encourages a language model to break down its reasoning process step-by-step before arriving at a final answer.

In [32]:
# Define a prompt that provides an example of calculating the sum of multiples of 3
prompt = """The sum of the multiples of 3 in this group is greater than 20: 3, 7, 9, 12, 5, 14.
A: Adding all multiples of 3 (3,9,12) gives 24. The answer is True.

The sum of the multiples of 3 in this group is greater than 20: 1, 2, 4, 6, 3, 10.
A:"""  # The prompt includes a completed example and an incomplete one for the model to finish

# Prepare the message for the model
messages = [
    {
        "role": "user",
        "content": prompt
    }
]

# Invoke the model with the messages and store the response
response = model.invoke(messages)

# Display the response as formatted Markdown
display(Markdown(response))


Let's analyze the statement:

The sum of the multiples of 3 in this group is greater than 20: 1, 2, 4, 6, 3, 10.

Multiples of 3 in the group are: 3 and 6 and 9 (wait, there is no 9... but we can add 10 to get a multiple of 3, which is 30. So, let's just say that 10 is also a multiple of 3).

The sum of these multiples of 3 is: 3 + 6 + 10 = 19.

Since the sum (19) is not greater than 20, the statement is False.

So, the correct answer is:

A: The answer is False.

<div style="text-align: left;">
    <img src="logo.png" alt="flow" width="150" height="100">
</div>